Note: datasets with citations generally come from here: 
https://blog.cambridgespark.com/50-free-machine-learning-datasets-sentiment-analysis-b9388f79c124
So, remember to cite if you use one

MORE NOTES TO SELF:
- Find average comment length, sentiment, score, etc.

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

First model: IMDB  
From:  
Maas, A., Daly, R., Pham, P., Huang, D., Ng, A. and Potts, C. (2011). Learning Word Vectors for Sentiment Analysis: Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies. [online] Portland, Oregon, USA: Association for Computational Linguistics, pp.142–150. Available at: http://www.aclweb.org/anthology/P11-1015.

This dataset comes in the form of a neg folder and a pos folder, each with comments in .txt format from IMDB. So, I'll need to put them in a more useful format.

In [12]:
import os

def folders_to_df(pos_path_as_str, neg_path_as_str):
    
    lst = []
    
    pos_path = os.fsencode(pos_path_as_str)
    neg_path = os.fsencode(neg_path_as_str)
    
    for file in os.listdir(pos_path):
        
        filepath = pos_path_as_str + "\\" + os.fsdecode(file)
        with open(filepath, "r", encoding="utf8") as f:
            body = f.read()
            lst.append([body, 1])
            
    for file in os.listdir(neg_path):
        
        filepath = neg_path_as_str + "\\" + os.fsdecode(file)
        with open(filepath, "r", encoding="utf8") as f:
            body = f.read()
            lst.append([body, -1])
            
    return pd.DataFrame(lst, columns = ["body", "sentiment"])

In [13]:
train_pos_path = r"C:\Users\Spencer\Desktop\sentiment_datasets\aclImdb\train\pos"
train_neg_path = r"C:\Users\Spencer\Desktop\sentiment_datasets\aclImdb\train\neg"

In [14]:
training_set = folders_to_df(train_pos_path, train_neg_path)

In [15]:
training_set.head()

,body,sentiment
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


The dataset is already split into train/test, but I'm combining them so I can define their ratio

In [19]:
test_pos_path = r"C:\Users\Spencer\Desktop\sentiment_datasets\aclImdb\test\pos"
test_neg_path = r"C:\Users\Spencer\Desktop\sentiment_datasets\aclImdb\test\neg"

testing_set = folders_to_df(test_pos_path, test_neg_path)

In [20]:
testing_set.head()

,body,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


In [26]:
imdb_dataset = training_set.append(testing_set, ignore_index=True)

In [28]:
imdb_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
body         50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


In [40]:
import string
from nltk.corpus import stopwords

def text_processor(text):
    punc_removed = ''.join([char for char in text if char not in string.punctuation])
    return [word.lower() for word in punc_removed.split() if word.lower() not in stopwords.words("english")]

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [44]:
count_vec = CountVectorizer(analyzer = text_processor)

In [46]:
count_vec = count_vec.fit_transform(imdb_dataset['body'])

In [48]:
from sklearn.model_selection import train_test_split

In [68]:
X_train, X_test, y_train, y_test = train_test_split(count_vec, imdb_dataset['sentiment'], test_size = 0.33, random_state = 2)

In [69]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [70]:
imdb_model = MultinomialNB().fit(X_train, y_train)

In [71]:
test_predictions = imdb_model.predict(X_test)

In [72]:
from sklearn.metrics import classification_report

In [73]:
print(classification_report(y_test, test_predictions))

              precision    recall  f1-score   support

          -1       0.84      0.88      0.86      8149
           1       0.88      0.84      0.86      8351

   micro avg       0.86      0.86      0.86     16500
   macro avg       0.86      0.86      0.86     16500
weighted avg       0.86      0.86      0.86     16500

